# 파트 5: Blob 지식 소스

파트 1-4에서는 사전 인덱싱된 데이터, SharePoint, 웹 소스를 사용했습니다. 파트 5에서는 Azure Blob Storage에서 문서를 업로드하고 자동으로 인덱싱하는 지식 소스를 생성합니다. 또한 두 가지 인덱싱 모드를 비교합니다: **최소(minimal)**(기본 콘텐츠 추출)와 **표준(standard)**(Azure AI Services를 사용한 고급 콘텐츠 이해).

## 단계 1: 환경 변수 로드

아래 셀을 실행하여 Azure 리소스의 구성을 로드하고, 생성된 **.venv(3.11.9)** 환경을 선택하세요.

문서 수집 및 벡터화에 필요한 blob 스토리지, AI 서비스, 임베딩 모델에 대한 추가 변수를 확인하세요. 이 모든 Azure 리소스는 `.env`에 사전 구성되어 있습니다.

> **⚠️ 문제 해결**
>
> 코드 셀이 멈추고 계속 돌아가면 노트북 상단 툴바에서 **Restart**를 선택하세요. 몇 번 시도해도 문제가 지속되면 VS Code를 완전히 닫고 다시 여세요.

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "upload-blob-knowledge-base-minimal"
standard_knowledge_base_name = "upload-blob-knowledge-base-standard"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large")

# Blob configuration
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
blob_container_name = os.environ["BLOB_CONTAINER_NAME"]
ai_services_endpoint = os.environ["AI_SERVICES_ENDPOINT"]
ai_services_key = os.environ["AI_SERVICES_KEY"]

blob_path = "../data/ai-search-data/blobdata/MSFT_cloud_architecture_zava.pdf"

print("Environment variables loaded")

Environment variables loaded


## 단계 2: Blob Storage에 문서 업로드

지식 소스를 생성하기 전에 blob 스토리지에 문서를 업로드해야 합니다. 아래 코드는 Zava의 클라우드 아키텍처와 민감도 수준별 데이터 분류 방법에 대한 정보가 포함된 `MSFT_cloud_architecture_zava.pdf`라는 PDF를 업로드합니다.

다음 단계에서 blob 지식 소스를 생성하면 스토리지에서 이 PDF를 자동으로 찾아 쿼리를 위해 인덱싱합니다.

In [ ]:
from azure.storage.blob import BlobServiceClient

blob_service_client = BlobServiceClient.from_connection_string(conn_str=blob_connection_string)
container_client = blob_service_client.get_container_client(blob_container_name)
blob_name = os.path.basename(blob_path)
blob_client = container_client.get_blob_client(blob_name)
if not blob_client.exists():
    with open(blob_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

print(f"Setup sample data in {blob_container_name}")

Setup sample data in documents


## 단계 3: 최소 추출로 Blob 지식 소스 생성

**AzureBlobKnowledgeSource**는 blob 스토리지에서 문서를 자동으로 인덱싱합니다. 이전에 사용한 소스와 달리, 이것은 문서를 수집하고 처리합니다.

아래 코드는 `content_extraction_mode`가 **minimal**인 지식 소스를 생성합니다. 이 모드는 깊은 시맨틱 이해 없이 문서를 빠르게 청크합니다. 임베딩 모델(`text-embedding-3-large`)이 벡터 검색을 위해 청크를 벡터화하는 데 사용되지만, 청킹 전략 자체는 기본적이고 빠릅니다.

>최소 인덱싱은 속도가 필요하고 간단한 문서가 있을 때 이상적입니다.

In [9]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureBlobKnowledgeSource, AzureBlobKnowledgeSourceParameters, AzureOpenAIVectorizerParameters, KnowledgeSourceAzureOpenAIVectorizer, KnowledgeSourceContentExtractionMode, KnowledgeSourceIngestionParameters

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

embedding_model = KnowledgeSourceAzureOpenAIVectorizer(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=azure_openai_endpoint,
        api_key=azure_openai_key,
        deployment_name=azure_openai_embedding_deployment,
        model_name=azure_openai_embedding_model_name
    )
)

knowledge_source = AzureBlobKnowledgeSource(
    name="upload-blob-knowledge-source-minimal",
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=blob_connection_string,
        container_name=blob_container_name,
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            embedding_model=embedding_model,
            content_extraction_mode=KnowledgeSourceContentExtractionMode.MINIMAL
        )
    )
)

index_client.create_or_update_knowledge_source(knowledge_source=knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'upload-blob-knowledge-source-minimal' created or updated successfully.


## 단계 4: 지식 소스 상태 확인

blob 지식 소스를 생성한 후 문서를 처리하는 데 시간이 필요합니다. 아래 코드는 인덱싱이 완료되었는지, 진행 중인지, 실패했는지 확인합니다.

`itemsUpdatesProcessed`가 1이면 단일 문서가 성공적으로 인덱싱되었음을 의미합니다. 인덱싱이 완료되면 다음 단계로 이동할 수 있습니다.

In [10]:
import json

status = index_client.get_knowledge_source_status(knowledge_source.name)

print(json.dumps(status.serialize(), indent=2))

{
  "synchronizationStatus": "active",
  "synchronizationInterval": "1d",
  "lastSynchronizationState": {
    "startTime": "2026-01-22T07:45:09.471Z",
    "endTime": "2026-01-22T07:45:14.577Z",
    "itemsUpdatesProcessed": 1,
    "itemsUpdatesFailed": 0,
    "itemsSkipped": 0
  },
  "statistics": {
    "totalSynchronization": 1,
    "averageSynchronizationDuration": "PT5.1065845S",
    "averageItemsProcessedPerSynchronization": 1
  }
}


## 단계 5: 지식 베이스 생성

blob 지식 소스가 문서를 인덱싱했으므로 이제 쿼리할 지식 베이스를 생성할 수 있습니다. 아래 코드는 앞서 생성한 blob 지식 소스를 사용하는 지식 베이스를 생성합니다.

이 지식 베이스는 `retrieval_reasoning_effort`도 "low"로 설정했습니다. 현재 가능한 가장 낮은 노력은 "minimal"이고 가장 높은 것은 "medium"입니다. "low" 노력은 여전히 쿼리 분해를 수행하지만 반복적 검색은 하지 않습니다.

In [11]:
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalLowReasoningEffort, KnowledgeRetrievalOutputMode, KnowledgeSourceReference

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    api_key=azure_openai_key,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
)

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name=knowledge_source.name)
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    retrieval_reasoning_effort=KnowledgeRetrievalLowReasoningEffort
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge base 'upload-blob-knowledge-base-minimal' created or updated successfully.


## 단계 6: 에이전틱 검색을 사용하여 Blob 지식 소스에서 결과 가져오기

아래 코드는 Zava의 데이터 민감도 분류 수준에 대한 PDF 문서를 쿼리합니다. 이는 에이전틱 검색이 blob 지식 소스와 어떻게 작동하는지 보여줍니다.

이 쿼리를 실행하면 지식 베이스가 질문을 분석하고, 집중된 하위 쿼리로 분해하며, blob 인덱싱된 콘텐츠를 동시에 검색하고, 시맨틱 랭킹을 사용하여 결과를 필터링하며, PDF 문서를 가리키는 인용과 함께 근거 있는 답변을 합성합니다.

In [12]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import AzureBlobKnowledgeSourceParams, KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)

blob_ks_params = AzureBlobKnowledgeSourceParams(
    knowledge_source_name=knowledge_source.name,
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="Zava 데이터 민감도 분류에는 어떤 단계가 있나요?")])
    ],
    knowledge_source_params=[
        blob_ks_params
    ],
    include_activity=True
)


result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

Zava의 데이터 민감도 분류는 세 가지 단계로 나뉩니다. 

1단계(낮은 비즈니스 가치): 일반 비즈니스 커뮤니케이션(이메일)과 행정, 영업, 지원 직원의 파일 등과 같은 데이터가 해당되며, 데이터는 암호화되어 인증된 사용자만 접근할 수 있습니다[ref_id:0].

2단계(중간 비즈니스 가치): 재무 및 법률 정보, 신제품 연구개발 데이터 등이 포함되며, 강력한 인증(예: SMS 검증이 포함된 다중 인증)과 데이터 유출 방지 기능이 적용됩니다[ref_id:0].

3단계(높은 비즈니스 가치): 고객 및 파트너의 개인 식별 정보, 제품 엔지니어링 사양, 독점 제조 기술 등이 해당되며, 최고 수준의 암호화, 스마트카드를 이용한 다중 인증, 세분화된 감사 및 경고가 적용됩니다[ref_id:0][ref_id:6].

## 단계 7: 응답, 참조, 활동 검토

아래 두 셀은 blob 지식 소스 쿼리에서 인용과 활동 로그를 보여줍니다.

참조는 질문에 답하기 위해 PDF에서 어떤 청크가 사용되었는지 보여줍니다.

활동 로그는 지식 베이스가 쿼리를 어떻게 처리하고 blob 인덱싱된 콘텐츠에서 정보를 검색했는지 보여줍니다.

In [13]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

[
  {
    "type": "azureBlob",
    "id": "0",
    "activity_source": 1,
    "source_data": {
      "uid": "b1b94a8afcf0_aHR0cHM6Ly9sYWI1MTFzdHhzbmJhNzRvN3p6Mm0uYmxvYi5jb3JlLndpbmRvd3MubmV0L2RvY3VtZW50cy9NU0ZUX2Nsb3VkX2FyY2hpdGVjdHVyZV96YXZhLnBkZg2_pages_17",
      "blob_url": "https://lab511stxsnba74o7zz2m.blob.core.windows.net/documents/MSFT_cloud_architecture_zava.pdf",
      "snippet": "No data sent across the Internet is in plain text form. Always use HTTPS connections, IPsec, or other end -to-end data \n\nencryption methods. \n\nEncryption for data at rest in \n\nthe cloud \n\n \n\nAll data stored on disks or elsewhere in the cloud must be in an encrypted form. \n\nACLs for least privilege \n\naccess \n\n \n\nAccount permissions to access resources in the cloud and what they are allowed to do must follow least-privilege guidelines. \n\n \n\nZava s data sensitivity classification \nUsing the information in Microsoft s Data Classification Toolkit, Zava performed an analysis of their

In [14]:
import pandas as pd

activity_types = [{"type": a.type} for a in result.activity]

df = pd.DataFrame(activity_types)

print("Activity Log Steps")
df

Activity Log Steps


,type
0,modelQueryPlanning
1,azureBlob
2,azureBlob
3,agenticReasoning
4,modelAnswerSynthesis


In [15]:
activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("Activity Details")
print(activity_content)

Activity Details
[
  {
    "id": 0,
    "type": "modelQueryPlanning",
    "elapsed_ms": 1527,
    "input_tokens": 1460,
    "output_tokens": 70
  },
  {
    "id": 1,
    "type": "azureBlob",
    "elapsed_ms": 630,
    "knowledge_source_name": "upload-blob-knowledge-source-minimal",
    "query_time": "2026-01-22T07:45:58.802Z",
    "count": 10,
    "azure_blob_arguments": {
      "search": "Zava \ub370\uc774\ud130 \ubbfc\uac10\ub3c4 \ubd84\ub958 \ub2e8\uacc4"
    }
  },
  {
    "id": 2,
    "type": "azureBlob",
    "elapsed_ms": 291,
    "knowledge_source_name": "upload-blob-knowledge-source-minimal",
    "query_time": "2026-01-22T07:45:59.094Z",
    "count": 10,
    "azure_blob_arguments": {
      "search": "Zava \ub370\uc774\ud130 \ubbfc\uac10\ub3c4 \ubd84\ub958 \ud504\ub85c\uc138\uc2a4"
    }
  },
  {
    "id": 3,
    "type": "agenticReasoning",
    "reasoning_tokens": 23742,
    "retrieval_reasoning_effort": {
      "kind": "low"
    }
  },
  {
    "id": 4,
    "type": "modelAnswerS

## 단계 8: 콘텐츠 이해와 함께 표준 추출 모드 사용

이전 단계에서 최소 추출 모드로 blob 지식 소스를 생성했습니다. 이제 더 깊은 콘텐츠 이해를 위해 Azure AI Services를 활용하는 **표준** 추출 모드를 사용하여 다른 blob 지식 소스를 생성합니다. 이 모드는 고급 청킹 전략, 시맨틱 추출, 복잡한 문서의 더 나은 처리를 제공합니다.

아래 코드는 `content_extraction_mode=STANDARD`를 추가하고 향상된 처리를 위해 Azure AI Services를 연결합니다.

>표준 추출은 시간이 더 걸리지만 문서 구조와 관계를 보존하는 더 높은 품질의 청크를 생성합니다.

In [44]:

from azure.search.documents.indexes.models import AIServices, KnowledgeSourceContentExtractionMode

standard_knowledge_source = AzureBlobKnowledgeSource(
    name="upload-blob-knowledge-source-standard",
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=blob_connection_string,
        container_name=blob_container_name,
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            embedding_model=embedding_model,
            ai_services=AIServices(uri=ai_services_endpoint, api_key=ai_services_key),
            content_extraction_mode=KnowledgeSourceContentExtractionMode.STANDARD
        )
    )
)

index_client.create_or_update_knowledge_source(knowledge_source=standard_knowledge_source)
print(f"Knowledge source '{standard_knowledge_source.name}' created or updated successfully.")

Knowledge source 'upload-blob-knowledge-source-standard' created or updated successfully.


## 단계 9: 표준 추출 상태 확인

아래 셀을 실행하여 표준 추출 진행 상황을 모니터링합니다. 이 모드는 Azure AI Services를 사용하여 문서 구조를 분석하고, 테이블을 인식하며, 지능형 청킹을 수행하므로 앞서 사용한 최소 추출 모드보다 시간이 더 걸립니다.

`itemsUpdatesProcessed`가 1이면 단일 문서가 성공적으로 인덱싱되었음을 의미합니다. 인덱싱이 완료되면 다음 단계로 이동할 수 있습니다.

In [37]:
import json

status = index_client.get_knowledge_source_status(standard_knowledge_source.name)

print(json.dumps(status.serialize(), indent=2))

{
  "synchronizationStatus": "active",
  "synchronizationInterval": "1d",
  "lastSynchronizationState": {
    "startTime": "2026-01-22T08:02:35.993Z",
    "endTime": "2026-01-22T08:02:40.985Z",
    "itemsUpdatesProcessed": 1,
    "itemsUpdatesFailed": 1,
    "itemsSkipped": 0
  },
  "statistics": {
    "totalSynchronization": 6,
    "averageSynchronizationDuration": "PT4.9051541S",
    "averageItemsProcessedPerSynchronization": 0
  }
}


## 단계 10: 표준 추출을 위한 지식 베이스 생성

이제 표준 추출 blob 지식 소스를 사용하는 지식 베이스를 생성합니다. 이 지식 베이스는 향상된 문서 처리와 개선된 청크 품질의 이점을 받게 됩니다.

아래 셀을 실행하여 표준 추출 소스로 지식 베이스를 생성합니다.

In [24]:
from azure.search.documents.indexes.models import KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference

standard_knowledge_base = KnowledgeBase(
    name=standard_knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name=standard_knowledge_source.name)
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS
)

index_client.create_or_update_knowledge_base(standard_knowledge_base)
print(f"Knowledge base '{standard_knowledge_base_name}' created or updated successfully.")

Knowledge base 'upload-blob-knowledge-base-standard' created or updated successfully.


## 단계 11: 표준 추출 지식 베이스 쿼리

Zava의 데이터 민감도 분류 수준에 대한 동일한 쿼리를 실행하지만, 이번에는 표준 추출 지식 베이스에 대해 실행합니다.

이 응답을 단계 6의 응답과 비교해보세요. 개선된 문서 처리로 인해 답변 품질, 완전성 또는 구성에서 차이를 발견할 수 있습니다.

In [29]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import AzureBlobKnowledgeSourceParams, KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest

standard_knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=standard_knowledge_base_name, credential=credential)

blob_ks_params = AzureBlobKnowledgeSourceParams(
    knowledge_source_name=standard_knowledge_source.name,
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="Zava 데이터 민감도 분류에는 어떤 단계가 있나요?")])
    ],
    knowledge_source_params=[
        blob_ks_params
    ],
    include_activity=True
)


result = standard_knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

귀하의 질문에 대한 관련 내용을 찾을 수 없습니다.

## 단계 12: 추출 결과 비교

아래 셀은 표준 추출 쿼리에서 인용을 보여줍니다.

이 참조를 단계 7의 참조와 비교하여 다른 추출 모드가 동일한 PDF 문서에서 청크 생성과 정보 검색에 어떻게 영향을 미치는지 확인하세요.

In [ ]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

## 요약

이제 blob 지식 소스를 경험하고 문서 처리를 위한 다양한 콘텐츠 추출 모드를 비교했습니다.

**기억해야 할 핵심 개념:**
- `AzureBlobKnowledgeSource`는 Azure Blob Storage에서 문서를 자동으로 인덱싱합니다
- **최소 추출**: 간단한 문서에 적합한 빠르고 기본적인 텍스트 추출
- **표준 추출**: 고급 문서 이해와 더 나은 청크 품질을 위해 Azure AI Services 사용
- 표준 추출은 테이블, 이미지 또는 복잡한 레이아웃이 있는 복잡한 문서에 유용합니다
- 두 모드 모두 blob 문서에서 검색 가능하고 벡터화된 청크를 생성합니다

### 다음 단계

➡️ [파트 6: 결합된 지식 소스](part6-combined-knowledge-source.ipynb)로 계속하여 단일 지식 베이스에서 검색 인덱스, 웹 URL, SharePoint, blob 스토리지를 동시에 쿼리하는 방법을 배웁니다.